In [2]:
import pyodbc
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

When the command above completes, the `pyodbc` and `pandas` library is loaded in your notebook.

## Identify the database connection properties

Connecting to SQL database server requires the following information:

*   Driver Name
*   Server
*   Database
*   User ID
*   User Password


In [3]:
#Replace the placeholder values with the actuals for your SQL Server connection properties
Server   = "HE170351"
Database = "LAB3"
UID      = " "
PWD      = " "

connection_str = ("Driver={SQL Server Native Client 11.0};"
                  "Server=" + Server + ";"
                  "Database=" + Database + ";"
                  "Trusted_Connection=yes;")

# connection_str = ("Driver={SQL Server Native Client 11.0};"
#                   "Server="+ Server + ";"
#                   "Database="+ Database + ";"
#                   "UID="+ UID + ";"
#                   "PWD="+ PWD + ";")

try:
    connection = pyodbc.connect(connection_str)
    print ("Connected to SQL database Server:", Server, "\nDatabase: " + Database)
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print ("Unable to connect: ", sqlstate)



Connected to SQL database Server: HE170351 
Database: LAB3


## Data structure understanding

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year.

Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012_.csv <a href='https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDev'>link<a/>

You are requested import this data set into SQL Server and and discover what's inside the data.

In [11]:
# You could run the following select query to first five records of INSTRUCTOR table
query_str = "SELECT TOP(10) * FROM Chicago_Public_Schools"

data = pd.read_sql(query_str, connection)
data.head()

DatabaseError: Execution failed on sql 'SELECT TOP(10) * FROM Chicago_Public_Schools': ('42S02', "[42S02] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid object name 'Chicago_Public_Schools'. (208) (SQLExecDirectW)")

## Task 1: Count the number of Elementary School, Middle School and High School in a query.



In [12]:
#Enter your code below
query = "SELECT SUM(CASE WHEN Elementary_Middle_or_High_School = 'ES' THEN 1 ELSE 0 END) AS Elementary_School, SUM(CASE WHEN Elementary_Middle_or_High_School = 'MS' THEN 1 ELSE 0 END) AS Middle_School,SUM(CASE WHEN Elementary_Middle_or_High_School = 'HS' THEN 1 ELSE 0 END) AS High_School FROM dbo.Chicago"
data = pd.read_sql(query, connection)
data

,Elementary_School,Middle_School,High_School
0,462,11,93


## Task 2: Find the school that has lowest average student attendance


In [13]:
#Enter your code below
query = "SELECT TOP 1 Name_of_School,Average_Student_Attendance FROM dbo.Chicago WHERE Average_Student_Attendance IS NOT NULL ORDER BY Average_Student_Attendance"
data = pd.read_sql(query, connection)
data

,Name_of_School,Average_Student_Attendance
0,Richard T Crane Technical Preparatory High School,57.900002


## Task 3: Find the average environment score


In [14]:
#Enter your code below
query = "SELECT AVG(Environment_Score) AS Average_Environment_Score FROM dbo.Chicago"
data = pd.read_sql(query, connection)
data

,Average_Environment_Score
0,47


## Task 4: Find the number of Elementary School in community area WEST TOWN


In [15]:
#Enter your code below
query = "SELECT COUNT(*) AS Number_Of_Elementary_West_Town FROM dbo.Chicago WHERE Community_Area_Name = 'WEST TOWN' AND Elementary_Middle_or_High_School = 'ES'"
data = pd.read_sql(query, connection)
data

,Number_Of_Elementary_West_Town
0,17


## Task 5: Count all the High School that has name start by letter ‘C’

In [16]:
#Enter your code below
query = "SELECT COUNT(*) AS Number_Of_High_School_Start_By_C FROM dbo.Chicago WHERE Name_of_School = 'C%' AND Elementary_Middle_or_High_School = 'HS'"
data = pd.read_sql(query, connection)
data

,Number_Of_High_School_Start_By_C
0,0


## Task 6: Find the top 10 Middle School has lowest Safety Score

In [17]:
#Enter your code below
query = "SELECT TOP 10 Name_of_School, Safety_Score FROM dbo.Chicago WHERE Elementary_Middle_or_High_School = 'MS' ORDER BY Safety_Score"
data = pd.read_sql(query, connection)
data

,Name_of_School,Safety_Score
0,Northwest Middle School,34
1,Irene C. Hernandez Middle School for the Advan...,34
2,Ames Middle School,38
3,Thurgood Marshall Middle School,39
4,Pershing West Middle School,46
5,Francisco I Madero Middle School,49
6,Williams Preparatory Academy Middle School,49
7,Logandale Middle School,56
8,Miriam G Canter Middle School,57
9,Evergreen Academy Middle School,60


## Task 7: Find all school has missing value in Leader Score column

In [18]:
#Enter your code below
query = "SELECT Name_of_School FROM dbo.Chicago WHERE Leaders_Score IS NULL"
data = pd.read_sql(query, connection)
data

,Name_of_School
0,Austin Business and Entrepreneurship Academy H...
1,William Jones College Preparatory High School
2,Walter Payton College Preparatory High School
3,North-Grand High School
4,Neal F Simeon Career Academy High School
...,...
266,South Shore Fine Arts Academy
267,Dr Jorge Prieto Math and Science
268,Skinner North
269,Mason High School


## Task 8: Update all missing value of environment score by mean of available data

In [22]:
#Enter your code below
query = "SELECT AVG(Leaders_Score) AS mean_score FROM Chicago"
data = pd.read_sql(query, connection)
data

,mean_score
0,49


In [23]:
update_query = f"UPDATE Chicago SET Leaders_Score = 49 WHERE Leaders_Score IS NULL"
cursor = connection.cursor()
cursor.execute(update_query)
connection.commit()

In [24]:
query = "SELECT Name_of_School FROM dbo.Chicago WHERE Leaders_Score IS NULL"
data = pd.read_sql(query, connection)
data

,Name_of_School


## Task 9: Using only one query to find out which ZIP code has the most and how many records have that ZIP code.

In [4]:
#Enter your code below
query = "SELECT TOP 1 t.ZIP_Code, COUNT(*) AS count_zipcode FROM dbo.Chicago t GROUP BY t.ZIP_Code ORDER BY count_zipcode DESC"
data = pd.read_sql(query, connection)
data

,ZIP_Code,count_zipcode
0,60623,35


## Task 10: Find out which address has the most concentration of schools. List all schools at that address.

In [ ]:
#Enter your code below


## Close database connection

In [ ]:
connection.close()

## Summary

In this tutorial you established a connection to a database instance of SQL Server Warehouse from a Python notebook using `pyodbc` API. Then created a table and insert a few rows of data into it. Then queried the data. You also retrieved the data into a pandas dataframe.
